In [1]:
import numpy as np
from math import ceil
import matplotlib.pyplot as plt
import sys
import re
import pandas as pd
import torch
import torch.nn as nn
import scipy
from scipy import ndimage
import torchvision
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
#from jupyterthemes import jtplot
import random
import cv2
from torchvision import models
from tqdm import tqdm
import new_transforms
import torchvision.transforms as transforms


data = pd.read_csv('fer2013/fer2013.csv')

In [2]:
print(len(data))
print(len(data.loc[0, 'pixels'].split(' ')))
#print(data.groupby('Usage').count()[['emotion']])

#Frequency of each label
print('0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral')
table = data.groupby('emotion').count()[['Usage']]
table['Pct'] = table['Usage']/table['Usage'].sum()
table['Pct'] = table['Pct'].map(lambda x: round(x, 3)*100)
table

35887
2304
0=Angry, 1=Disgust, 2=Fear, 3=Happy, 4=Sad, 5=Surprise, 6=Neutral


,Usage,Pct
emotion,,
0,4953,13.8
1,547,1.5
2,5121,14.3
3,8989,25.0
4,6077,16.9
5,4002,11.2
6,6198,17.3


## 1) Converting to numpy datasets

In [3]:
#imgSize %input the the model
imgSize=299
trans = transforms.Compose([transforms.ToPILImage(),
                            new_transforms.Resize((imgSize, imgSize)),
                            transforms.RandomHorizontalFlip(),
                            new_transforms.RandomRotate(),
                            transforms.ToTensor(),
                            transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [4]:
def convert_to_numpy(data):
    X = data[:,1]
    X = np.asarray([np.asarray(X[i].split(" ")) for i in range(X.shape[0])])
    X = np.asarray([X[i].reshape(48,48).astype(int) for i in range(X.shape[0])])
    y = data[:,0]
    return (X,y)

In [5]:
train = data[data['Usage'] == 'Training']
train_X, train_Y = convert_to_numpy(train.values)
valid = data[data['Usage'] == 'PrivateTest']
valid_X, valid_Y = convert_to_numpy(valid.values)
test = data[data['Usage'] == 'PublicTest']
test_X, test_Y = convert_to_numpy(test.values)

## 2) Function to extract the batches from the dataset

In [6]:
def data_iter(x, y, batch_size):
    dataset_size = x.shape[0]
    start = -1 * batch_size
    order = list(range(dataset_size))
    random.shuffle(order)

    while True:
        start += batch_size
        if start > dataset_size - batch_size:
            break   
        batch_indices = order[start:start + batch_size]
        yield np.asarray([x[index] for index in batch_indices]) ,np.asarray([y[index] for index in batch_indices])

## 3) Model 

## CNN Implementation

### 1) First implementation (Basic)

In [7]:
class CNN(nn.Module):
    """
    CNN model
    """
       
    def __init__(self, kernel_size, num_labels, n_layers=1, dropout=0.1):
       
        """
        @param vocab_size: size of the vocabulary. 
        @param emb_dim: size of the word embedding
        """
        super(CNN,self).__init__()

        self.conv1 = nn.Sequential(
                            nn.Conv2d(1, 10, kernel_size=kernel_size, stride=1, padding=1),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2),
                            )
        self.conv2 = nn.Sequential(
                            nn.Conv2d(10, 20, kernel_size=kernel_size, stride=1, padding=1),
                            nn.ReLU(),
                            nn.MaxPool2d(kernel_size=2),
                            )
        self.out = nn.Linear(20*12*12, num_labels)
        self.dropout = nn.Dropout(dropout)
    
    def forward(self, x):
        x = x.unsqueeze(1) # (N,Ci,W,D)
        x = self.conv1(x)
        x = self.conv2(x)
        x = x.view(x.size(0), -1) # Note that normally ppl dont use dropout in CNN
        x = self.out(x)
        return torch.nn.functional.softmax(x)

## 4) Training stage setup

In [8]:
def early_stop(val_acc_history, t=2, required_progress=0.001):    
    cnt = 0 # initialize the count --> to store count of cases where difference in
                                    #  accuracy is less than required progress.
    
    if(len(val_acc_history) > 0): # if list has size > 0 
        for i in range(t): # start the loop
            index = len(val_acc_history) - (i+1) # start from the last term in list and move to the left
            if (index >= 1): # to check if index != 0 --> else we can't compare to previous value
                if (abs(val_acc_history[index] - val_acc_history[index-1]) < required_progress):
                    cnt += 1 # increase the count value
                else:
                    break # break if difference is grea-ter 
    
    if(cnt != t): # if count is equal to t, return True
        return False
    else:
        return True
    

def train(train_X, train_Y, valid_X, valid_Y, optimizer, model, batch_size, num_epochs, criterion, to_Add_Softmax=False, is_inception=False):
    losses = []
    total_batches = int(train_X.shape[0]/ batch_size)
    validation_losses = []
    
    eval_every = 10
    print_every = 10
    validate_every = int((eval_every/100)*total_batches)
    show_every = int((print_every/100)*total_batches)
    
    for epoch in range(1, num_epochs+1):
        stop_training = False
        train_data = data_iter(train_X, train_Y, batch_size)
        for i, (x,y) in enumerate(train_data):
            x = Variable(torch.from_numpy(x).type(torch.FloatTensor))
            y = Variable(torch.from_numpy(y).type(torch.LongTensor))
            model.train(True)
            optimizer.zero_grad()
            outputs = model(x)
            if is_inception == True:
                outputs = outputs[0]
            if to_Add_Softmax == True:
                outputs = nn.functional.softmax(outputs)
            loss = criterion(outputs, y)
            losses.append(loss.data[0])
            loss.backward()


            optimizer.step()
            
            if (i+1)%validate_every == 0:
                valid_loss_temp = []
                valid_data = data_iter(valid_X, valid_Y, batch_size)
                for j, (v_x, v_y) in enumerate(valid_data):
                    v_x = Variable(torch.from_numpy(v_x).type(torch.FloatTensor))
                    v_y = Variable(torch.from_numpy(v_y).type(torch.LongTensor))
                    model.eval()
                    val_outputs = model(v_x)
                    eval_loss = criterion(val_outputs, v_y)
                    valid_loss_temp.append(eval_loss.data[0])
                validation_losses.append(np.mean(valid_loss_temp))
                stop_training = early_stop(validation_losses, 3)
                
            if stop_training:
                print("earily stop triggered")
                break
            if (i+1) % show_every == 0:
                print('Epoch: [{0}/{1}], Step: [{2}/{3}], Train loss: {4}, Validation loss:{5}'.format(
                           epoch, num_epochs, i+1, total_batches, np.mean(losses)/(total_batches*epoch), np.mean(np.array(validation_losses))))
        if stop_training == True:
            break

## 5) Training the model

### 1) first model

In [9]:
num_labels = 7
num_epochs = 5
learning_rate = 0.01
kernel_size = 3
batch_size = 80

#model = CNN( kernel_size, num_labels, n_layers=1, dropout=0.1)
criterion = nn.CrossEntropyLoss()
#optimizer = optim.Adam(model.parameters(), lr=learning_rate)

#train(train_X, train_Y, valid_X, valid_Y, optimizer, model, batch_size, num_epochs, criterion, False, False)

### 2) second model

#### 2.1) Zoom the image and convert 1 channel to 3 channel dataset

In [10]:
def to_rgb1a(data, w, h):

    R, _, _ = data.shape
    temp = np.zeros((R, 3, w, h))
    
    for i in tqdm(range(R)):
        im = data[0]
        ret = np.empty((3, w, h), dtype=np.uint8)
        im = cv2.resize(im.astype(np.uint8), (w, h), interpolation=cv2.INTER_LINEAR)
        ret[0, :, :] =  ret[1, :, :] =  ret[2, :, :] =  im
        temp[i] = trans(ret).numpy()
        data = np.delete(data, 0, 0)
    
    return temp

#### Experimenting with a smaller dataset (as running imagenet architectures locally is very time consuming)

In [11]:
old_train_X = train_X
train_X = to_rgb1a(old_train_X[0:1000], 299, 299).astype(np.uint8)
old_valid_X = valid_X
valid_X = to_rgb1a(old_valid_X[0:500], 299, 299).astype(np.uint8)
old_test_X = test_X
test_X = to_rgb1a(old_test_X[0:500], 299, 299).astype(np.uint8)

100%|██████████| 500/500 [00:02<00:00, 190.40it/s]


#### 2.2) Using the pretrained models

In [12]:
### Still some modification needs to be made

alexnet = models.alexnet(pretrained=True)
num_ftrs = alexnet.classifier

## Modifying and training only the last layer
for param in alexnet.parameters():
    param.requires_grad = False

alexnet.classifier._modules['6'] = nn.Linear(4096, num_labels)
# num_ftrs = alexnet.classifier[6].in_features
# alexnet.classifier[6].out_features = num_labels
for param in alexnet.classifier[6].parameters():
    param.requires_grad = True
    
#optimizer = optim.Adam(filter(lambda p: p.requires_grad, alexnet.parameters()), alexnet.classifier.parameters())
optimizer = optim.Adam(alexnet.classifier[6].parameters(), lr=0.0001)
num_epochs = 1



In [13]:
train(train_X, train_Y, valid_X, valid_Y, optimizer, alexnet, batch_size, num_epochs, criterion, to_Add_Softmax=True, is_inception=False)

RuntimeError: invalid argument 2: size '[80 x 9216]' is invalid for input of with 1310720 elements at /Users/soumith/miniconda2/conda-bld/pytorch_1503975723910/work/torch/lib/TH/THStorage.c:41

#### 3) resnet

In [14]:
resnet = models.resnet50(pretrained=True)
# freeze all model parameters
for param in resnet.parameters():
    param.requires_grad = False

# new final layer with 7 classes
num_ftrs = resnet.fc.in_features
resnet.fc = torch.nn.Linear(num_ftrs, num_labels)
optimizer = optim.Adam(resnet.fc.parameters(), lr=0.0001)
num_epochs = 1

train(train_X, train_Y, valid_X, valid_Y, optimizer, resnet, batch_size, num_epochs, criterion, to_Add_Softmax=True, is_inception=False)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /Users/bilals01/.torch/models/resnet50-19c8e357.pth
100%|██████████| 102502400/102502400 [00:45<00:00, 2268094.32it/s]


Epoch: [1/1], Step: [1/12], Train loss: 0.16237634420394897, Validation loss:2.110840678215027


KeyboardInterrupt: 

#### 4) Inception

##### Inception requires input size to be (299, 299)

In [ ]:
train_X = to_rgb1a(old_train_X[0:1000], 299, 299)
valid_X = to_rgb1a(old_valid_X[0:500], 299, 299)
test_X = to_rgb1a(old_test_X[0:500], 299, 299)

In [ ]:
incptn = models.inception_v3(pretrained=True)
# freeze all model parameters
for param in incptn.parameters():
    param.requires_grad = False

# new final layer with 7 classes
num_ftrs = incptn.fc.in_features
incptn.fc = torch.nn.Linear(num_ftrs, num_labels)
optimizer = optim.Adam(incptn.fc.parameters(), lr=0.0001)
num_epochs = 1

train(train_X, train_Y, valid_X, valid_Y, optimizer, incptn, batch_size, num_epochs, criterion, to_Add_Softmax=True, is_inception=True)

## 6) Calculating accuracy on test set

In [ ]:
#test_output = model(Variable(torch.from_numpy(test_X).type(torch.FloatTensor)))
resnet.train(False)
test_output = resnet(Variable(torch.from_numpy(test_X).type(torch.FloatTensor)))
pred_y = torch.max(test_output, 1)[1].data.numpy().squeeze()
accuracy = sum(pred_y == test_Y[0:500])/len(test_Y[0:500])
print(accuracy)